In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c exploring-predictive-health-factors
!unzip /content/exploring-predictive-health-factors.zip
!rm /content/exploring-predictive-health-factors.zip

Mounted at /content/drive
  0% 0.00/5.28k [00:00<?, ?B/s]
100% 5.28k/5.28k [00:00<00:00, 13.1MB/s]


In [ ]:
!pip install autogluon.tabular[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.2/266.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB

## Import Packages

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, GammaRegressor, Ridge
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
import matplotlib.pyplot as plt
import xgboost as xgb

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# from keras_tuner import HyperModel
# from keras_tuner import RandomSearch
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import layers, models

from autogluon.tabular import TabularDataset, TabularPredictor

## Load Data

In [ ]:
train = pd.read_csv('/content/train.csv', index_col = 0)
test = pd.read_csv('/content/test.csv', index_col = 0)
sub = pd.read_csv('/content/sample_submission.csv')

## EDA

In [ ]:
train.isna().sum()

,0
Age,1
Weight_kg,2
PCOS,0
Hormonal_Imbalance,2
Hyperandrogenism,3
Hirsutism,5
Conception_Difficulty,1
Insulin_Resistance,1
Exercise_Frequency,2
Exercise_Type,2


In [ ]:
test.isna().sum()

,0
Age,1
Weight_kg,2
Hormonal_Imbalance,3
Hyperandrogenism,1
Hirsutism,2
Conception_Difficulty,2
Insulin_Resistance,1
Exercise_Frequency,1
Exercise_Type,1
Exercise_Duration,0


## AutoML Model

In [ ]:
label = 'PCOS'
metric = 'roc_auc'
time_limit = 600

excluded_model_types = ['KNN']

In [ ]:
predictor = TabularPredictor(label, eval_metric=metric).fit(train, auto_stack=True,excluded_model_types=excluded_model_types,
                                                            time_limit=time_limit, presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20250125_201829"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       11.29 GB / 12.67 GB (89.0%)
Disk Space Avail:   75.79 GB / 107.72 GB (70.4%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be f

(_ray_fit pid=2247) [1000]	valid_set's binary_logloss: 0.176563


(_ray_fit pid=2332) 
(_ray_fit pid=2332) 
(_ray_fit pid=2341) 
(_ray_fit pid=2341) 
(_dystack pid=1912) 	0.8873	 = Validation score   (roc_auc)
(_dystack pid=1912) 	20.52s	 = Training   runtime
(_dystack pid=1912) 	0.07s	 = Validation runtime
(_ray_fit pid=2341) /usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning:  [repeated 3x across cluster]
(_ray_fit pid=2341) Dask dataframe query planning is disabled because dask-expr is not installed. [repeated 3x across cluster]
(_ray_fit pid=2341) You can install it with `pip install dask[dataframe]` or `conda install dask`. [repeated 3x across cluster]
(_ray_fit pid=2341) This will raise in a future version. [repeated 3x across cluster]
(_ray_fit pid=2341)   warnings.warn(msg, FutureWarning) [repeated 3x across cluster]
(_dystack pid=1912) Fitting model: LightGBM_BAG_L1 ... Training model for up to 67.50s of the 115.53s of remaining time.
(_dystack pid=1912) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with P

In [ ]:
predictor.feature_importance(data=train)

Computing feature importance via permutation shuffling for 12 features using 210 rows with 5 shuffle sets...
	161.67s	= Expected runtime (32.33s per shuffle set)
	41.62s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
Hyperandrogenism,0.075133,0.014175,0.000145,5,0.104318,0.045947
Exercise_Type,0.050610,0.009158,0.000123,5,0.069465,0.031754
Hormonal_Imbalance,0.042683,0.016359,0.002151,5,0.076366,0.009000
Insulin_Resistance,0.033139,0.011172,0.001341,5,0.056143,0.010135
Conception_Difficulty,0.016516,0.003973,0.000373,5,0.024697,0.008336
Hirsutism,0.007582,0.003441,0.003943,5,0.014666,0.000498
Exercise_Benefit,0.004374,0.002036,0.004314,5,0.008567,0.000181
Weight_kg,0.003314,0.002503,0.020763,5,0.008467,-0.001839
Exercise_Frequency,0.002757,0.000891,0.001146,5,0.004592,0.000922
Age,0.002041,0.001961,0.040214,5,0.006079,-0.001996


In [ ]:
predictor.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.895811,roc_auc,1.631015,308.932779,0.000728,0.117911,3,True,20
1,WeightedEnsemble_L2,0.891437,roc_auc,0.758877,156.681583,0.000867,0.093163,2,True,11
2,CatBoost_BAG_L2,0.886930,roc_auc,1.036613,228.051815,0.101274,50.992705,2,True,16
3,LightGBMXT_BAG_L2,0.880435,roc_auc,1.035481,198.021883,0.100142,20.962773,2,True,12
4,CatBoost_BAG_L1,0.878844,roc_auc,0.099988,72.888699,0.099988,72.888699,1,True,5
5,LightGBMXT_BAG_L1,0.876524,roc_auc,0.082231,21.006332,0.082231,21.006332,1,True,1
6,ExtraTreesGini_BAG_L2,0.874536,roc_auc,1.158290,177.894647,0.222950,0.835537,2,True,17
7,ExtraTreesEntr_BAG_L2,0.869234,roc_auc,1.054039,177.765414,0.118699,0.706304,2,True,18
8,LightGBM_BAG_L2,0.865589,roc_auc,1.063654,200.891321,0.128315,23.832211,2,True,13
9,ExtraTreesEntr_BAG_L1,0.862540,roc_auc,0.105866,0.778396,0.105866,0.778396,1,True,7


In [ ]:
# Make preds
y_pred = predictor.predict_proba(test,as_multiclass=False)
y_pred.to_csv('/content/drive/MyDrive/Output/sub2.csv')

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
y_pred

,PCOS
ID,
0,0.031129
1,0.291001
2,0.234596
3,0.550943
4,0.446732
...,...
140,0.181685
141,0.055282
142,0.584274


In [ ]:
# Submit
!kaggle competitions submit -c exploring-predictive-health-factors -f /content/drive/MyDrive/Output/sub2.csv -m "all feats"

100% 2.02k/2.02k [00:00<00:00, 10.4kB/s]
Successfully submitted to Exploring Predictive Health Factors